In [1]:
!pip install dlib


In [2]:
pip install face-detection

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install face-recognition

Note: you may need to restart the kernel to use updated packages.


# 

In [ ]:
pip install mtcnn


In [5]:
import cv2
import numpy as np
from mtcnn import MTCNN
import face_recognition

# Initialize the MTCNN face detector
detector = MTCNN()

# Step 1: Read and encode multiple input images
def load_and_encode_images(image_paths):
    encodings = []
    names = []
    
    for image_path in image_paths:
        # Load and convert the image to RGB
        input_image = cv2.imread(image_path)
        rgb_input_image = cv2.cvtColor(input_image, cv2.COLOR_BGR2RGB)
        
        # Detect faces using MTCNN
        faces = detector.detect_faces(rgb_input_image)
        
        # If no face detected, skip this image
        if len(faces) == 0:
            print(f"No face detected in the image: {image_path}")
            continue
        
        # Extract face locations and encodings
        face_locations = [(face['box'][1], face['box'][0] + face['box'][2], face['box'][1] + face['box'][3], face['box'][0]) for face in faces]
        input_face_encodings = face_recognition.face_encodings(rgb_input_image, face_locations)
        
        # Save the first encoding (assuming one face per image)
        encodings.append(input_face_encodings[0])
        names.append(image_path.split("/")[-1].split(".")[0])
    
    return encodings, names

# Step 2: Process video and search for multiple faces
def search_faces_in_video(video_path, known_face_encodings, known_face_names, threshold=0.6, process_every_nth_frame=3):
    # Open the video file
    video_capture = cv2.VideoCapture(video_path)

    frame_count = 0  # Track the frame count

    while video_capture.isOpened():
        # Read frame by frame
        ret, frame = video_capture.read()
        if not ret:
            print("End of video reached or error reading video.")
            break  # End of video

        frame_count += 1
        # Process every nth frame to reduce CPU load
        if frame_count % process_every_nth_frame != 0:
            continue
        
        # Optionally resize frame for faster processing
        small_frame = cv2.resize(frame, (0, 0), fx=0.5, fy=0.5)  # Downscaling by 50%
        rgb_frame = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)

        # Detect faces in the current frame using MTCNN
        faces_in_frame = detector.detect_faces(rgb_frame)
        if len(faces_in_frame) == 0:
            continue  # Skip if no faces were found

        # Extract face locations
        frame_face_locations = [(face['box'][1], face['box'][0] + face['box'][2], face['box'][1] + face['box'][3], face['box'][0]) for face in faces_in_frame]
        frame_face_encodings = face_recognition.face_encodings(rgb_frame, frame_face_locations)

        for (top, right, bottom, left), face_encoding in zip(frame_face_locations, frame_face_encodings):
            # Compare the detected face with all known face encodings
            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)

            if len(face_distances) == 0:
                continue  # Skip if no known faces are available

            # Find the best match
            best_match_index = np.argmin(face_distances)
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding, tolerance=threshold)
            
            if matches[best_match_index]:
                # Label the face with the corresponding known face name
                name = known_face_names[best_match_index]

                # Draw a box around the face (adjust for the downscaled frame)
                top, right, bottom, left = [int(x * 2) for x in [top, right, bottom, left]]  # Scale up
                cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
                cv2.putText(frame, name, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            else:
                # Label as unknown
                top, right, bottom, left = [int(x * 2) for x in [top, right, bottom, left]]  # Scale up
                cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
                cv2.putText(frame, "Unknown", (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

        # Display the frame with bounding boxes
        cv2.imshow('Video', frame)
        
        # Handle keyboard interrupt and break if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            print("Video stopped by user.")
            break

    # Release the video capture object and close the display window
    video_capture.release()
    cv2.destroyAllWindows()

# Main function
if __name__ == '__main__':
    # Paths to multiple images and video
    input_image_paths = [r'C:\Users\swath\OneDrive\Desktop\face_detection\woman.png',
                         r'C:\Users\swath\OneDrive\Desktop\face_detection\man.png']  # Add paths to all images
    video_path = r"C:\Users\swath\OneDrive\Desktop\face_detection\3205621-hd_1920_1080_25fps.mp4"

    # Step 1: Load and encode the multiple input images
    known_face_encodings, known_face_names = load_and_encode_images(input_image_paths)
    print("Loaded images and encodings")
    
    # Step 2: Search for these faces in the video
    search_faces_in_video(video_path, known_face_encodings, known_face_names)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 392ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 161ms/step
Loaded images and encodings
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━

# 